# Building the Roman Road Network using itiner-e dataset
This notebook builds the Roman Road Network (RRN) using functions from `itinerex_clean_network.py` and the data originally released as part of the `[Nature](https://www.nature.com/articles/s41597-025-06140-z)` publication. 

It produces:
- `roads_before_cleaning.html`: raw roads from GeoJSON (display only)
- `roads_after_cleaning.html`: cleaned, noded, degree-2 simplified network with **node_id** and **edge_id** shown on the map
- `G_clean_weighted_with_ids.pkl`: pickle containing the cleaned weighted graph + node/edge tables for mapping analysis results back to geometry/CRS.

All metric operations run in **EPSG:3395**; maps are displayed in **EPSG:4326**.

In [5]:
from __future__ import annotations
import pickle
from pathlib import Path
import numpy as np
import geopandas as gpd
import networkx as nx
import folium
import importlib
import itinerex_clean_network as icn
import pandas as pd
import matplotlib.colors as mcolors
from shapely.geometry import Point
importlib.reload(icn)

# -----------------------------
# Configuration
# -----------------------------
GEOJSON_PATH = Path("17122148/itinere_roads.geojson")

# These tolerances are in meters (EPSG:3395).
SNAP_TOL_M = 240.0
MERGE_DEG1_TOL_M = 240.0

# Constant-speed time weights (matches earlier baseline).
BASE_SPEED_KMH = 2.0
WEIGHT_MODE = "time"  # sets edge attribute 'weight' = travel time (seconds)

OUT_BEFORE_HTML = Path("roads_before_cleaning.html")
OUT_AFTER_HTML = Path("roads_after_cleaning.html")
OUT_PICKLE = Path("G_clean_weighted_with_ids.pkl")


def _union_all(gs):
    """GeoSeries union helper compatible with Shapely 2 / older GeoPandas."""
    try:
        return gs.union_all()
    except Exception:
        return gs.unary_union


def _node_id_mapping(G: nx.Graph):
    """Assign stable node_ids (1..N) by sorting coordinates."""
    nodes = [(float(x), float(y)) for (x, y) in G.nodes()]
    nodes_sorted = sorted(nodes, key=lambda xy: (xy[0], xy[1]))
    node_to_id = {xy: i + 1 for i, xy in enumerate(nodes_sorted)}
    id_to_node = {i: xy for xy, i in node_to_id.items()}
    return node_to_id, id_to_node


def _edge_id_mapping(G: nx.Graph, node_to_id: dict[tuple[float, float], int]):
    """Assign stable edge_ids (1..M) by sorting undirected (u_id, v_id)."""
    pairs = []
    for u, v in G.edges():
        uid = int(node_to_id[(float(u[0]), float(u[1]))])
        vid = int(node_to_id[(float(v[0]), float(v[1]))])
        a, b = (uid, vid) if uid <= vid else (vid, uid)
        pairs.append((a, b, (u, v)))
    pairs_sorted = sorted(pairs, key=lambda t: (t[0], t[1]))
    edge_key_to_id = {(a, b): i + 1 for i, (a, b, _) in enumerate(pairs_sorted)}
    return edge_key_to_id


def graph_to_tables(G: nx.Graph, crs_3395: str = "EPSG:3395"):
    """Return (nodes_gdf_3395, edges_gdf_3395, nodes_gdf_4326, edges_gdf_4326)."""
    node_to_id, id_to_node = _node_id_mapping(G)
    edge_key_to_id = _edge_id_mapping(G, node_to_id)

    # Nodes
    deg = dict(G.degree())
    rows_n = []
    for xy, node_id in node_to_id.items():
        x, y = float(xy[0]), float(xy[1])
        rows_n.append({
            "node_id": int(node_id),
            "x": x,
            "y": y,
            "degree": int(deg.get(xy, 0)),
        })
    nodes_3395 = gpd.GeoDataFrame(
        rows_n,
        geometry=gpd.points_from_xy([r["x"] for r in rows_n], [r["y"] for r in rows_n]),
        crs=crs_3395,
    )
    nodes_4326 = nodes_3395.to_crs(epsg=4326)

    # Edges (requires 'geometry' on graph edges)
    rows_e = []
    for u, v, d in G.edges(data=True):
        u = (float(u[0]), float(u[1]))
        v = (float(v[0]), float(v[1]))
        uid = int(node_to_id[u])
        vid = int(node_to_id[v])
        a, b = (uid, vid) if uid <= vid else (vid, uid)
        edge_id = int(edge_key_to_id[(a, b)])
        rows_e.append({
            "edge_id": edge_id,
            "u_id": uid,
            "v_id": vid,
            "weight": float(d.get("weight", 1.0)),
            "time_s": float(d.get("time_s", d.get("weight", np.nan))),
            "dist_m": float(d.get("dist_m", np.nan)),
            "geometry": d.get("geometry"),
        })
    edges_3395 = gpd.GeoDataFrame(rows_e, crs=crs_3395)
    edges_3395 = edges_3395[edges_3395.geometry.notna() & ~edges_3395.geometry.is_empty].copy()
    edges_4326 = edges_3395.to_crs(epsg=4326)

    return nodes_3395, edges_3395, nodes_4326, edges_4326, node_to_id, id_to_node, edge_key_to_id


def save_before_map(roads_4326: gpd.GeoDataFrame, out_html: Path, simplify_tol_m: float = 50.0):
    """Save a lightweight BEFORE map.

    The raw roads are often too large to embed feature-by-feature in a Folium HTML.
    We project to EPSG:3395, simplify in meters, dissolve into one geometry, then map that.
    """
    if roads_4326 is None or len(roads_4326) == 0:
        raise ValueError("No roads to plot")

    roads_m = roads_4326.to_crs(epsg=3395)
    simp = roads_m.geometry.simplify(float(simplify_tol_m), preserve_topology=True)
    merged_m = _union_all(simp)
    merged_4326 = gpd.GeoDataFrame({"geometry": [merged_m]}, crs="EPSG:3395").to_crs(epsg=4326)

    center = merged_4326.geometry.iloc[0].centroid
    m = folium.Map(location=[float(center.y), float(center.x)], zoom_start=6, tiles="CartoDB positron")
    folium.GeoJson(merged_4326.__geo_interface__, name="Roads (raw, simplified)").add_to(m)
    folium.LayerControl(collapsed=False).add_to(m)
    m.save(str(out_html))
    return out_html


def save_after_map(edges_4326: gpd.GeoDataFrame, nodes_4326: gpd.GeoDataFrame, out_html: Path):
    if (edges_4326 is None or len(edges_4326) == 0) and (nodes_4326 is None or len(nodes_4326) == 0):
        raise ValueError("No cleaned nodes/edges to plot")
    if edges_4326 is not None and len(edges_4326):
        center = _union_all(edges_4326.geometry).centroid
    else:
        center = _union_all(nodes_4326.geometry).centroid
    m = folium.Map(location=[float(center.y), float(center.x)], zoom_start=6, tiles="CartoDB positron")

    fg_edges = folium.FeatureGroup(name="Edges (cleaned)", show=True)
    fg_nodes = folium.FeatureGroup(name="Nodes (cleaned)", show=True)
    m.add_child(fg_edges)
    m.add_child(fg_nodes)

    use_edges = edges_4326.copy()
    for col in ["edge_id", "u_id", "v_id"]:
        use_edges[col] = use_edges[col].astype(int)
    if "time_s" in use_edges.columns:
        use_edges["time_s"] = use_edges["time_s"].astype(float)
    if "dist_m" in use_edges.columns:
        use_edges["dist_m"] = use_edges["dist_m"].astype(float)
    folium.GeoJson(
        use_edges.__geo_interface__,
        name="Edges (cleaned)",
        tooltip=folium.GeoJsonTooltip(
            fields=["edge_id", "u_id", "v_id", "time_s", "dist_m"],
            aliases=["edge_id", "u_id", "v_id", "time_s", "dist_m"],
            sticky=True,
        ),
    ).add_to(fg_edges)

    for row in nodes_4326.itertuples(index=False):
        lat = float(row.geometry.y)
        lon = float(row.geometry.x)
        folium.CircleMarker(
            location=(lat, lon),
            radius=3,
            color="#2ca02c",
            fill=True,
            fill_opacity=0.9,
            tooltip=f"node_id={int(row.node_id)} deg={int(row.degree)}",
        ).add_to(fg_nodes)

    folium.LayerControl(collapsed=False).add_to(m)
    m.save(str(out_html))
    return out_html

# 1) Load raw roads + BEFORE map
Loads the input roads GeoJSON, projects to EPSG:3395 for metric operations, and saves a raw “before cleaning” HTML map.

In [6]:
# Load (keep a 4326 copy for mapping)
roads_4326 = gpd.read_file(GEOJSON_PATH)
roads_4326 = roads_4326[roads_4326.geometry.notna() & ~roads_4326.geometry.is_empty].copy()
roads_4326 = roads_4326[roads_4326.geometry.geom_type.isin(["LineString", "MultiLineString"])].copy()
roads_4326 = roads_4326.reset_index(drop=True)

# Project for cleaning operations
roads_3395 = roads_4326.to_crs(epsg=3395)
print("roads_4326:", len(roads_4326), "features")
print("roads_3395 CRS:", roads_3395.crs)

out_before = save_before_map(roads_4326, OUT_BEFORE_HTML)
print("Saved:", out_before)

roads_4326: 14769 features
roads_3395 CRS: EPSG:3395
Saved: roads_before_cleaning.html


# 2) Clean rebuild + AFTER map + export
Builds a cleaned topology from the raw GeoJSON using `itinerex_clean_network.build_clean_network_from_geojson()`, assigns stable `node_id`/`edge_id`, saves an “after cleaning” HTML map with those IDs, and exports everything as a pickle.

In [7]:
# Build cleaned network (EPSG:3395 metric topology)
res = icn.build_clean_network_from_geojson(
    str(GEOJSON_PATH),
    snap_tol_m=float(SNAP_TOL_M),
    snap_method="endpoints",
    diagnose=False,
    merge_deg1_tol_m=float(MERGE_DEG1_TOL_M),
    merge_deg1_min_samples=2,
    simplify_deg2=True,
    base_speed_kmh=float(BASE_SPEED_KMH),
    weight_mode=str(WEIGHT_MODE),
    show_progress=True,
 )

print(res.diagnostics)

G_clean: nx.Graph = res.graph
segments_clean_3395: gpd.GeoDataFrame = res.segments_m
noded_lines_clean_3395: gpd.GeoDataFrame = res.noded_lines_m

print("G_clean:", G_clean.number_of_nodes(), "nodes;", G_clean.number_of_edges(), "edges")
print("Example edge attrs:", next(iter(G_clean.edges(data=True)))[2])

# Assign stable IDs and create node/edge tables (both 3395 + 4326)
nodes_clean_3395, edges_clean_3395, nodes_clean_4326, edges_clean_4326, node_to_id, id_to_node, edge_key_to_id = graph_to_tables(
    G_clean, crs_3395="EPSG:3395"
 )

# Attach node_id/edge_id onto the graph too (for routing + analysis)
for xy, node_id in node_to_id.items():
    if xy in G_clean:
        G_clean.nodes[xy]["node_id"] = int(node_id)

for u, v, d in G_clean.edges(data=True):
    uid = int(node_to_id[(float(u[0]), float(u[1]))])
    vid = int(node_to_id[(float(v[0]), float(v[1]))])
    a, b = (uid, vid) if uid <= vid else (vid, uid)
    d["edge_id"] = int(edge_key_to_id[(a, b)])
    d["u_id"] = uid
    d["v_id"] = vid

print("nodes_clean_3395:", len(nodes_clean_3395), "rows")
print("edges_clean_3395:", len(edges_clean_3395), "rows")

[1/10] load...
[2/10] prepare...
[3/10] snap...
Snapping line endpoints to nearby linework
[4/10] node...
[5/10] segmentize...
Converting lines to segments
[6/10] graph...
Building graph from segments
[7/10] merge_deg1...
[8/10] simplify_deg2...
Simplifying degree-2 nodes
{'snap_tol_m': 240.0, 'near_miss_before': {'skipped': 1}, 'near_miss_after': {'skipped': 1}, 'noded_lines': 16851, 'segments': 1001619, 'graph_nodes': 8460, 'graph_edges': 12849, 'deg2_nodes': 0, 'deg1_nodes': 833, 'timings_s': {'load': 1.8017390000022715, 'prepare': 0.027291874997899868, 'snap': 1.506065875000786, 'node': 1.026009500004875, 'segmentize': 4.102779790999193, 'graph': 10.99082887500117, 'merge_deg1': 16.527879458000825, 'simplify_deg2': 14.977518125000643}, 'snap_method': 'endpoints', 'diagnose': False, 'diagnose_max_endpoints': None, 'merge_deg1': {'enabled': 1, 'deg1_before': 998, 'deg1_after': 829, 'clusters': 81, 'merged_nodes': 173}, 'base_speed_kmh': 2.0, 'weight_mode': 'time'}
G_clean: 8460 nodes

In [8]:
# Save AFTER map (cleaned network) with node_id / edge_id tooltips
out_after = save_after_map(edges_clean_4326, nodes_clean_4326, OUT_AFTER_HTML)
print("Saved:", out_after)

# # Display inline map object
# m_after = folium.Map(location=[float(nodes_clean_4326.geometry.y.mean()), float(nodes_clean_4326.geometry.x.mean())], zoom_start=6, tiles="CartoDB positron")
# folium.GeoJson(
#     edges_clean_4326.__geo_interface__,
#     name="Edges (cleaned)",
#     tooltip=folium.GeoJsonTooltip(fields=["edge_id", "u_id", "v_id", "time_s", "dist_m"], sticky=True),
# ).add_to(m_after)
# for row in nodes_clean_4326.itertuples(index=False):
#     folium.CircleMarker(
#         location=(float(row.geometry.y), float(row.geometry.x)),
#         radius=3,
#         color="#2ca02c",
#         fill=True,
#         fill_opacity=0.9,
#         tooltip=f"node_id={int(row.node_id)} deg={int(row.degree)}",
#     ).add_to(m_after)
# folium.LayerControl(collapsed=False).add_to(m_after)
# m_after

Saved: roads_after_cleaning.html


In [9]:
def graph_stats(G: nx.Graph, weight: str | None = "weight"):
    """Basic graph stats + (optional) weighted degree (strength)."""
    n = int(G.number_of_nodes())
    m = int(G.number_of_edges())
    avg_deg = (2.0 * m / n) if n else 0.0

    if weight is None:
        wdeg = dict(G.degree())
    else:
        if m and not any((weight in d) for _, _, d in G.edges(data=True)):
            print(f"WARNING: edge attribute '{weight}' not found on any edge; weighted degree will equal unweighted degree.")
        wdeg = dict(G.degree(weight=weight))
    avg_wdeg = float(np.mean(list(wdeg.values()))) if n else 0.0

    comps = list(nx.connected_components(G))
    num_cc = int(len(comps))
    largest_cc_size = int(max((len(c) for c in comps), default=0))
    if largest_cc_size:
        largest_nodes = max(comps, key=len)
        G_lcc = G.subgraph(largest_nodes)
        lcc_edges = int(G_lcc.number_of_edges())
        lcc_frac = float(largest_cc_size / n)
    else:
        lcc_edges = 0
        lcc_frac = 0.0

    return {
        "nodes": n,
        "edges": m,
        "avg_degree": float(avg_deg),
        "avg_weighted_degree": float(avg_wdeg),
        "connected_components": num_cc,
        "largest_component_nodes": largest_cc_size,
        "largest_component_edges": lcc_edges,
        "largest_component_fraction_of_nodes": float(lcc_frac),
    }

stats = graph_stats(G_clean, weight="weight")
for k, v in stats.items():
    print(f"{k:35s}: {v}")

nodes                              : 8460
edges                              : 12849
avg_degree                         : 3.0375886524822695
avg_weighted_degree                : 167930.63592041272
connected_components               : 12
largest_component_nodes            : 7612
largest_component_edges            : 11693
largest_component_fraction_of_nodes: 0.8997635933806146


In [10]:
# Export cleaned weighted network + mapping tables

# NOTE on portability:
# Pickling raw NetworkX graphs can be brittle across Python/NetworkX versions.
# This export is reconstruction-friendly: it stores node/edge tables + an edge list.

edges_export_3395 = edges_clean_3395.copy()
try:
    edges_export_3395["geometry_wkt"] = edges_export_3395.geometry.to_wkt()
except Exception:
    edges_export_3395["geometry_wkt"] = edges_export_3395.geometry.apply(lambda g: g.wkt if g is not None else None)

nodes_export_3395 = nodes_clean_3395.copy()
try:
    nodes_export_3395["geometry_wkt"] = nodes_export_3395.geometry.to_wkt()
except Exception:
    nodes_export_3395["geometry_wkt"] = nodes_export_3395.geometry.apply(lambda g: g.wkt if g is not None else None)

export = {
    "crs_metric": "EPSG:3395",
    "crs_map": "EPSG:4326",
    "params": {
        "geojson": str(GEOJSON_PATH),
        "snap_tol_m": float(SNAP_TOL_M),
        "merge_deg1_tol_m": float(MERGE_DEG1_TOL_M),
        "base_speed_kmh": float(BASE_SPEED_KMH),
        "weight_mode": str(WEIGHT_MODE),
    },
    # Full GeoDataFrames (with shapely geometries)
    "nodes_3395": nodes_export_3395,
    "edges_3395": edges_export_3395,
    "nodes_4326": nodes_clean_4326,
    "edges_4326": edges_clean_4326,
    # Reconstruction-friendly tables (no shapely dependency required if using WKT)
    "node_table": nodes_export_3395.drop(columns=["geometry"], errors="ignore"),
    "edge_table": edges_export_3395.drop(columns=["geometry"], errors="ignore"),
    # Stable mappings
    "node_to_id": node_to_id,
    "id_to_node": id_to_node,
}

with open(OUT_PICKLE, "wb") as f:
    pickle.dump(export, f, protocol=pickle.HIGHEST_PROTOCOL)

print("Saved:", OUT_PICKLE.resolve())

Saved: /Users/nk821/Documents/GitHub/itinereX/G_clean_weighted_with_ids.pkl


# 4) Province Overlay & Sub-Networks

Loads Roman province polygons from the shapefile in `provinces/provinces.shp`, of Roman provinces in 200 CE (peak Roman Empire under the Severan dynasty), based on 'roman-empire-ce-200-provinces.geojson' published by The Ancient World Mapping Centre and corrected by Adam Pazout in 2023. https://github.com/AWMC/geodata/tree/master/Cultural-Data/political_shading/roman_empire_ce_200_provinces. The provinces are overlaid on the roman road network and corresponding provicial road networks are constructed. 

**Outputs:**
| File | Contents |
|---|---|
| `roads_with_provinces_nodes.html` | Maps marking each province and the road network in each province where nodes are intersections and edges are the road geometries |
| `G_clean_weighted_edges_with_province.csv` | Edge list for the weighted network: `edge_id`, (`u_id`,`v_id`), `time_s`, and province assignment |
| `crossing_edges.geojson` | Edges whose **endpoints** fall in **different** provinces (excluded from sub-networks) |
| `outside_provinces_edges.geojson` | Edges that cannot be assigned to any province (excluded from sub-networks) |
| `province_subnetworks.pkl` | Dict with `province_subgraphs` (NetworkX), `province_edge_tables` (GeoDataFrames), `crossing_edge_ids`, `outside_edge_ids`, `edge_province_map` |

**Classification logic:**
- **Province assignment:** assign each edge to the (most specific) province containing its midpoint; if the midpoint is not inside any polygon, fall back to the province with the longest overlap length.
- **Crossing edges (excluded):** an edge is marked crossing if its **start** and **end** points fall in **different** provinces.
- **Outside edges (excluded):** an edge is outside if it cannot be assigned to any province by midpoint or overlap fallback.

In [ ]:
# --- Province Overlay & Sub-Networks (Shapefile only) ---
from pathlib import Path
import pickle
import folium
import geopandas as gpd
import matplotlib.colors as mcolors
import networkx as nx
import pandas as pd
from shapely.geometry import Point

PROVINCES_PATH          = Path("provinces/provinces.shp")
OUT_PROVINCE_MAP_NODES  = Path("roads_with_provinces_nodes.html")
OUT_WEIGHTED_EDGES_PROV = Path("G_clean_weighted_edges_with_province.csv")
OUT_CROSSING_EDGES      = Path("crossing_edges.geojson")
OUT_OUTSIDE_EDGES       = Path("outside_provinces_edges.geojson")
OUT_PROVINCE_NETWORKS   = Path("province_subnetworks.pkl")
IN_CLEAN_PICKLE = Path("G_clean_weighted_with_ids.pkl")
_required_vars = ["edges_clean_3395", "edges_clean_4326", "nodes_clean_4326", "G_clean"]
_missing = [v for v in _required_vars if v not in globals()]
if _missing:
    if not IN_CLEAN_PICKLE.exists():
        raise RuntimeError(
            "Missing in-kernel variables: " + ", ".join(_missing) + "\n"
            "Run the network cleaning cells above first, or ensure G_clean_weighted_with_ids.pkl exists."
        )
    data = pickle.load(IN_CLEAN_PICKLE.open("rb"))
    # Tables for mapping + province overlay
    if "nodes_clean_3395" not in globals():
        nodes_clean_3395 = data["nodes_3395"]
    if "edges_clean_3395" not in globals():
        edges_clean_3395 = data["edges_3395"]
    if "nodes_clean_4326" not in globals():
        nodes_clean_4326 = data["nodes_4326"]
    if "edges_clean_4326" not in globals():
        edges_clean_4326 = data["edges_4326"]
    node_to_id = data.get("node_to_id")
    id_to_node = data.get("id_to_node")

    # Reconstruct G_clean (only if missing) from saved id mappings + tables
    if "G_clean" not in globals():
        if id_to_node is None:
            raise RuntimeError("Pickle is missing id_to_node; can't reconstruct G_clean.")
        node_table = data.get("node_table")
        edge_table = data.get("edge_table")
        if node_table is None or edge_table is None:
            raise RuntimeError("Pickle missing node_table/edge_table; can't reconstruct G_clean.")

        G_clean = nx.Graph()
        for row in node_table.itertuples(index=False):
            nid = int(getattr(row, "node_id"))
            nkey = id_to_node.get(nid)
            if nkey is None:
                continue
            G_clean.add_node(
                nkey,
                node_id=nid,
                x=float(getattr(row, "x")),
                y=float(getattr(row, "y")),
                degree=int(getattr(row, "degree")),
            )
        for row in edge_table.itertuples(index=False):
            ukey = id_to_node.get(int(getattr(row, "u_id")))
            vkey = id_to_node.get(int(getattr(row, "v_id")))
            if ukey is None or vkey is None:
                continue
            G_clean.add_edge(
                ukey,
                vkey,
                edge_id=int(getattr(row, "edge_id")),
                weight=float(getattr(row, "weight")),
                time_s=float(getattr(row, "time_s")),
                dist_m=float(getattr(row, "dist_m")),
            )
        print(f"[Loaded from {IN_CLEAN_PICKLE}] Reconstructed G_clean: {G_clean.number_of_nodes()} nodes; {G_clean.number_of_edges()} edges")

# ── 1. Load province polygons (shapefile) ─────────────────────────────────────
_provinces_raw = gpd.read_file(PROVINCES_PATH)
_provinces_raw = _provinces_raw[_provinces_raw.geometry.notna() & ~_provinces_raw.geometry.is_empty].copy()

if _provinces_raw.crs is None:
    raise ValueError("Province shapefile has no CRS; please ensure .prj is present.")

# Prefer explicit field name; otherwise try to auto-detect a reasonable name column
_name_col = None
if "province" in _provinces_raw.columns:
    _name_col = "province"
else:
    _candidates = [c for c in _provinces_raw.columns if c != "geometry" and ("name" in str(c).lower() or "prov" in str(c).lower())]
    if _candidates:
        _name_col = _candidates[0]
    else:
        _obj_cols = [c for c in _provinces_raw.columns if c != "geometry" and str(_provinces_raw[c].dtype) == "object"]
        if _obj_cols:
            _name_col = _obj_cols[0]

if _name_col is None:
    raise ValueError(f"Couldn't find a province name column in shapefile. Columns: {list(_provinces_raw.columns)}")

_provinces_raw["name"] = _provinces_raw[_name_col].astype(str)
provinces_4326 = _provinces_raw[["name", "geometry"]].copy()

# Dissolve to one row per province name (handles multi-part provinces cleanly)
provinces_4326 = provinces_4326.dissolve(by="name", as_index=False)

# Make geometry valid where possible (avoid topology errors during intersections)
try:
    provinces_4326["geometry"] = provinces_4326.geometry.make_valid()
except Exception:
    provinces_4326["geometry"] = provinces_4326.buffer(0)

provinces_4326 = provinces_4326.to_crs(epsg=4326)
print(f"Loaded {len(provinces_4326)} province polygons from {PROVINCES_PATH}")
print(f"Unique province names: {provinces_4326['name'].nunique()}")
print(f"Province CRS: {provinces_4326.crs}")

# ── 2. Assign edges to provinces (most-specific midpoint) + crossing detection ──
provinces_3395 = provinces_4326.to_crs(epsg=3395).copy()
provinces_3395["_prov_area"] = provinces_3395.geometry.area.astype(float)
edges_3395 = edges_clean_3395[["edge_id", "geometry"]].copy()
edges_3395["edge_id"] = edges_3395["edge_id"].astype(int)

def _as_linestring(geom):
    if geom is None or geom.is_empty:
        return geom
    gt = getattr(geom, "geom_type", None)
    if gt == "LineString":
        return geom
    if gt == "MultiLineString":
        try:
            return max(list(geom.geoms), key=lambda g: g.length)
        except Exception:
            return geom
    return geom

def _midpoint(geom):
    geom = _as_linestring(geom)
    try:
        return geom.interpolate(0.5, normalized=True)
    except Exception:
        return geom.centroid

def _endpoints(geom):
    geom = _as_linestring(geom)
    try:
        xs, ys = geom.coords[0]
        xe, ye = geom.coords[-1]
        return Point(xs, ys), Point(xe, ye)
    except Exception:
        c = geom.centroid
        return c, c

def _pick_most_specific(join_df, group_cols, name_col, area_col):
    if join_df is None or len(join_df) == 0:
        return {}
    use = join_df[join_df[name_col].notna()].copy()
    if len(use) == 0:
        return {}
    use[area_col] = use[area_col].astype(float)
    use = use.sort_values([*group_cols, area_col], ascending=True)
    picked = use.groupby(group_cols)[name_col].first()
    return picked.to_dict()

# --- Midpoint-based province assignment (most-specific) ---
midpoints_3395 = gpd.GeoDataFrame(
    {"edge_id": edges_3395["edge_id"].values},
    geometry=[_midpoint(g) for g in edges_3395["geometry"]],
    crs="EPSG:3395",
)
mid_join = gpd.sjoin(
    midpoints_3395,
    provinces_3395[["name", "_prov_area", "geometry"]],
    how="left",
    predicate="intersects",
)
name_col_mid = "name_right" if "name_right" in mid_join.columns else "name"
area_col_mid = "_prov_area_right" if "_prov_area_right" in mid_join.columns else "_prov_area"
mid_best = _pick_most_specific(mid_join, ["edge_id"], name_col_mid, area_col_mid)
unresolved_ids = set(edges_3395["edge_id"].tolist()) - set(mid_best.keys())
print(f"\nEdge assignment")
print(f"  Resolved by midpoint (most-specific): {len(mid_best)}")
print(f"  Needing overlap fallback           : {len(unresolved_ids)}")

# --- Fallback: longest overlap length (tie-break by smallest polygon area) ---
fallback_map = {}  # edge_id -> province_name
if unresolved_ids:
    unresolved_edges = edges_3395[edges_3395["edge_id"].isin(unresolved_ids)].copy()
    prov_sindex = provinces_3395.sindex
    prov_areas = provinces_3395["_prov_area"].astype(float).tolist()
    eps = 1e-6
    for _, erow in unresolved_edges.iterrows():
        eid = int(erow["edge_id"])
        egeom = _as_linestring(erow["geometry"])
        best_len = 0.0
        best_area = float("inf")
        best_prov = None
        candidates = list(prov_sindex.intersection(egeom.bounds))
        for ci in candidates:
            pgeom = provinces_3395.geometry.iloc[int(ci)]
            pname = provinces_3395["name"].iloc[int(ci)]
            parea = float(prov_areas[int(ci)])
            try:
                inter = egeom.intersection(pgeom)
                if inter is None or inter.is_empty:
                    continue
                ilen = float(inter.length)
                if ilen <= 0:
                    continue
                if (ilen > best_len + eps) or (abs(ilen - best_len) <= eps and parea < best_area):
                    best_len = ilen
                    best_area = parea
                    best_prov = pname
            except Exception:
                continue
        if best_prov is not None:
            fallback_map[eid] = str(best_prov)
print(f"  Resolved by overlap fallback         : {len(fallback_map)}")

edge_province_map_all = dict(mid_best)
edge_province_map_all.update(fallback_map)

# --- Crossing detection: endpoints in different provinces (most-specific) ---
pt_rows = []
for eid, geom in zip(edges_3395["edge_id"].tolist(), edges_3395["geometry"].tolist()):
    spt, ept = _endpoints(geom)
    pt_rows.append({"edge_id": int(eid), "which": "start", "geometry": spt})
    pt_rows.append({"edge_id": int(eid), "which": "end",   "geometry": ept})
endpoints_3395 = gpd.GeoDataFrame(pt_rows, crs="EPSG:3395")
end_join = gpd.sjoin(
    endpoints_3395,
    provinces_3395[["name", "_prov_area", "geometry"]],
    how="left",
    predicate="intersects",
)
name_col_end = "name_right" if "name_right" in end_join.columns else "name"
area_col_end = "_prov_area_right" if "_prov_area_right" in end_join.columns else "_prov_area"
picked_end = _pick_most_specific(end_join, ["edge_id", "which"], name_col_end, area_col_end)
start_map = {eid: picked_end.get((eid, "start")) for eid in edges_3395["edge_id"].tolist()}
end_map   = {eid: picked_end.get((eid, "end"))   for eid in edges_3395["edge_id"].tolist()}

crossing_edge_ids = set()
for eid in edges_3395["edge_id"].tolist():
    sprov = start_map.get(int(eid))
    eprov = end_map.get(int(eid))
    if (sprov is None) or (eprov is None):
        continue
    if str(sprov) != str(eprov):
        crossing_edge_ids.add(int(eid))

# Final within-province assignment excludes crossing edges
edge_province_map = {eid: prov for eid, prov in edge_province_map_all.items() if int(eid) not in crossing_edge_ids}

all_edge_ids = set(edges_3395["edge_id"].tolist())
outside_edge_ids = all_edge_ids - set(edge_province_map_all.keys())

# Province-counts table for exports
province_counts_rows = []
for eid in sorted(all_edge_ids):
    eid = int(eid)
    if eid in crossing_edge_ids:
        provs = []
        if start_map.get(eid) is not None:
            provs.append(str(start_map[eid]))
        if end_map.get(eid) is not None:
            provs.append(str(end_map[eid]))
        provs = sorted(set(provs))
        province_counts_rows.append({"edge_id": eid, "province_list": provs, "n_provinces": len(provs)})
    elif eid in edge_province_map_all:
        province_counts_rows.append({"edge_id": eid, "province_list": [str(edge_province_map_all[eid])], "n_provinces": 1})
    else:
        province_counts_rows.append({"edge_id": eid, "province_list": [], "n_provinces": 0})
province_counts = pd.DataFrame(province_counts_rows)

print(f"\nEdge classification")
print(f"  Total edges                  : {len(edges_clean_4326)}")
print(f"  Within-province (kept)       : {len(edge_province_map)}")
print(f"  Crossing border (excluded)   : {len(crossing_edge_ids)}")
print(f"  Outside all polygons         : {len(outside_edge_ids)}")

# ── 3. Export weighted edge list with province assignment ─────────────────────
_edge_cols = ["edge_id", "u_id", "v_id", "time_s", "dist_m", "weight"]
edges_for_export = edges_clean_3395.copy()
keep_cols = [c for c in _edge_cols if c in edges_for_export.columns]
if "edge_id" not in keep_cols:
    raise RuntimeError("edges_clean_3395 is missing edge_id; can't export weighted edge list")
edges_for_export = edges_for_export[keep_cols].copy()
edges_for_export["edge_id"] = edges_for_export["edge_id"].astype(int)
if "u_id" in edges_for_export.columns:
    edges_for_export["u_id"] = edges_for_export["u_id"].astype(int)
if "v_id" in edges_for_export.columns:
    edges_for_export["v_id"] = edges_for_export["v_id"].astype(int)

edges_for_export["province"] = edges_for_export["edge_id"].map(edge_province_map_all)
edges_for_export["province_start"] = edges_for_export["edge_id"].map(start_map)
edges_for_export["province_end"] = edges_for_export["edge_id"].map(end_map)
edges_for_export["is_crossing"] = edges_for_export["edge_id"].isin(crossing_edge_ids)
edges_for_export["is_outside"] = edges_for_export["edge_id"].isin(outside_edge_ids)
edges_for_export.to_csv(OUT_WEIGHTED_EDGES_PROV, index=False)
print(f"\nSaved weighted edge list → {OUT_WEIGHTED_EDGES_PROV} ({len(edges_for_export)} edges)")

# ── 4. Save crossing & outside edges ─────────────────────────────────────────
edges_map = edges_clean_4326.copy()
edges_map["edge_id"] = edges_map["edge_id"].astype(int)
edges_map["province"] = edges_map["edge_id"].map(edge_province_map)

prov_info = province_counts.set_index("edge_id")[["province_list", "n_provinces"]]

crossing_edges_4326 = edges_map[edges_map["edge_id"].isin(crossing_edge_ids)].copy()
ce_export = crossing_edges_4326.join(prov_info, on="edge_id")
ce_export["province_list"] = ce_export["province_list"].apply(lambda x: ";".join(x) if isinstance(x, list) else "")
ce_export.to_file(str(OUT_CROSSING_EDGES), driver="GeoJSON")
print(f"\nSaved crossing edges          → {OUT_CROSSING_EDGES}  ({len(ce_export)} edges)")

outside_edges_4326 = edges_map[edges_map["edge_id"].isin(outside_edge_ids)].copy()
outside_edges_4326.to_file(str(OUT_OUTSIDE_EDGES), driver="GeoJSON")
print(f"Saved outside-province edges  → {OUT_OUTSIDE_EDGES}   ({len(outside_edges_4326)} edges)")

# ── 5. Build per-province sub-networks ───────────────────────────────────────
province_names = sorted(set(edge_province_map.values()))
print(f"\nProvinces with assigned edges: {len(province_names)}")

province_subgraphs = {}
province_edge_tables = {}

for pname in province_names:
    prov_edge_ids = {eid for eid, pn in edge_province_map.items() if pn == pname}
    sub_edges = [
        (u, v, d) for u, v, d in G_clean.edges(data=True)
        if int(d.get("edge_id", -1)) in prov_edge_ids
]
    all_nodes = {n for u, v, _ in sub_edges for n in (u, v)}
    G_sub = nx.Graph()
    G_sub.add_nodes_from([(n, G_clean.nodes[n]) for n in all_nodes])
    G_sub.add_edges_from(sub_edges)
    province_subgraphs[pname] = G_sub
    province_edge_tables[pname] = edges_map[edges_map["edge_id"].isin(prov_edge_ids)].copy()

print(f"\n{'Province':<35} {'Nodes':>6}  {'Edges':>6}  {'Components':>10}")
print("-" * 65)
for pname, Gsub in province_subgraphs.items():
    n_comp = nx.number_connected_components(Gsub)
    print(f"{pname:<35} {Gsub.number_of_nodes():>6}  {Gsub.number_of_edges():>6}  {n_comp:>10}")

with open(OUT_PROVINCE_NETWORKS, "wb") as f:
    pickle.dump(
        {
            "province_subgraphs": province_subgraphs,
            "province_edge_tables": province_edge_tables,
            "crossing_edge_ids": list(crossing_edge_ids),
            "outside_edge_ids": list(outside_edge_ids),
            "edge_province_map": edge_province_map,
        },
        f,
        protocol=pickle.HIGHEST_PROTOCOL,
    )
print(f"\nSaved province sub-networks → {OUT_PROVINCE_NETWORKS}")

# ── 6. Map (only: roads_with_provinces_nodes.html) ───────────────────────────
_palette = list(mcolors.TABLEAU_COLORS.values()) + [
    "#e377c2", "#7f7f7f", "#bcbd22", "#17becf", "#aec7e8", "#ffbb78",
    "#98df8a", "#ff9896", "#c5b0d5", "#c49c94", "#f7b6d2", "#c7c7c7",
]
prov_colour = {row["name"]: _palette[i % len(_palette)] for i, row in provinces_4326.iterrows()}

# Simplify province boundaries for web display only (keeps HTML size down)
PROVINCE_SIMPLIFY_TOL_M = 5000.0
provinces_map_4326 = provinces_3395[["name", "geometry"]].copy()
provinces_map_4326["geometry"] = provinces_map_4326.geometry.simplify(PROVINCE_SIMPLIFY_TOL_M, preserve_topology=True)
provinces_map_4326 = provinces_map_4326.to_crs(epsg=4326)
try:
    provinces_map_4326["geometry"] = provinces_map_4326.geometry.make_valid()
except Exception:
    provinces_map_4326["geometry"] = provinces_map_4326.buffer(0)

# Center (based on edges)
_edges_3395_tmp = edges_map.to_crs(epsg=3395)
center_3395 = _edges_3395_tmp.dissolve().centroid.iloc[0]
center_4326 = gpd.GeoDataFrame(geometry=[center_3395], crs="EPSG:3395").to_crs(epsg=4326)
center_latlon = [float(center_4326.geometry.y.iloc[0]), float(center_4326.geometry.x.iloc[0])]

# Precompute edge subsets used for mapping
within_ids = set(edge_province_map.keys())
within_edges = edges_map[edges_map["edge_id"].isin(within_ids)].copy()
within_edges["_color"] = within_edges["province"].map(lambda p: prov_colour.get(str(p), "#1f77b4"))
crossing_edges = edges_map[edges_map["edge_id"].isin(set(crossing_edge_ids))].copy()
outside_edges  = edges_map[edges_map["edge_id"].isin(set(outside_edge_ids))].copy()

# Minimise properties embedded in the HTML GeoJSON (keeps file sizes sane)
within_edges   = within_edges[["edge_id", "province", "_color", "geometry"]].copy()
crossing_edges = crossing_edges[["edge_id", "geometry"]].copy()
outside_edges  = outside_edges[["edge_id", "geometry"]].copy()

def _add_provinces(m):
    fg = folium.FeatureGroup(name="Provinces", show=True)
    prov_gdf = provinces_map_4326 if "provinces_map_4326" in globals() else provinces_4326

    def _style(feat):
        pname = str(feat.get("properties", {}).get("name", ""))
        c = prov_colour.get(pname, "#888888")
        return {
            "fillColor": c, "color": c,
            "weight": 1.5, "fillOpacity": 0.18,
        }

    folium.GeoJson(
        prov_gdf[["name", "geometry"]].__geo_interface__,
        style_function=_style,
        tooltip=folium.GeoJsonTooltip(fields=["name"], aliases=["province"], sticky=True),
    ).add_to(fg)
    fg.add_to(m)

def _add_roads_within(m):
    fg = folium.FeatureGroup(name="Roads – inside province", show=True)
    folium.GeoJson(
        within_edges.__geo_interface__,
        style_function=lambda feat: {
            "color": feat["properties"].get("_color", "#1f77b4"),
            "weight": 2.5,
            "opacity": 0.9,
        },
        tooltip=folium.GeoJsonTooltip(fields=["edge_id", "province"], aliases=["edge_id", "province"], sticky=True),
    ).add_to(fg)
    fg.add_to(m)

def _add_roads_crossing(m):
    fg = folium.FeatureGroup(name="Roads – crossing border (excluded)", show=True)
    folium.GeoJson(
        crossing_edges.__geo_interface__,
        style_function=lambda feat: {
            "color": "#d62728",
            "weight": 2.0,
            "opacity": 0.85,
            "dashArray": "6,4",
        },
        tooltip=folium.GeoJsonTooltip(fields=["edge_id"], aliases=["edge_id"], sticky=True),
    ).add_to(fg)
    fg.add_to(m)

def _add_roads_outside(m):
    fg = folium.FeatureGroup(name="Roads – outside all provinces", show=False)
    folium.GeoJson(
        outside_edges.__geo_interface__,
        style_function=lambda feat: {
            "color": "#aaaaaa",
            "weight": 1.0,
            "opacity": 0.5,
        },
        tooltip=folium.GeoJsonTooltip(fields=["edge_id"], aliases=["edge_id"], sticky=True),
    ).add_to(fg)
    fg.add_to(m)

def _add_nodes(m):
    nodes = nodes_clean_4326
    if nodes.crs is None:
        nodes = nodes.set_crs("EPSG:4326")
    else:
        nodes = nodes.to_crs(epsg=4326)
    coords = [[float(p.y), float(p.x)] for p in nodes.geometry if p is not None and not p.is_empty]
    fg = folium.FeatureGroup(name="Network nodes (G_clean)", show=True)
    for lat, lon in coords:
        folium.CircleMarker(
            location=[lat, lon],
            radius=3,
            weight=3,
            color="#2ca02c",
            opacity=1.0,
            fill=True,
            fill_color="#2ca02c",
            fill_opacity=0.9,
        ).add_to(fg)
    fg.add_to(m)

m_prov_nodes = folium.Map(location=center_latlon, zoom_start=6, tiles="CartoDB positron")
_add_provinces(m_prov_nodes)
_add_roads_within(m_prov_nodes)
_add_roads_crossing(m_prov_nodes)
_add_roads_outside(m_prov_nodes)
_add_nodes(m_prov_nodes)
folium.LayerControl(collapsed=False).add_to(m_prov_nodes)
m_prov_nodes.save(str(OUT_PROVINCE_MAP_NODES))
print(f"Saved province overlay + nodes → {OUT_PROVINCE_MAP_NODES}")

Loaded 57 province polygons from provinces/provinces.shp
Unique province names: 57
Province CRS: EPSG:4326

Edge assignment
  Resolved by midpoint (most-specific): 12791
  Needing overlap fallback           : 58
  Resolved by overlap fallback         : 19

Edge classification
  Total edges                  : 12849
  Within-province (kept)       : 12173
  Crossing border (excluded)   : 637
  Outside all polygons         : 39

Saved weighted edge list → G_clean_weighted_edges_with_province.csv (12849 edges)

Saved crossing edges          → crossing_edges.geojson  (637 edges)
Saved outside-province edges  → outside_provinces_edges.geojson   (39 edges)

Provinces with assigned edges: 57

Province                             Nodes   Edges  Components
-----------------------------------------------------------------
Achaia                                 363     525           3
Aegyptus                               176     265           1
Aemilia (Regio VIII)                    28      35  